In [1]:
# import libraries
import mne
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from mne.time_frequency import tfr_morlet, tfr_array_morlet
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import BettiCurve, PersistenceEntropy, Amplitude

import argparse

# Define parameters
epoch_length = 5  # in seconds
overlap = 4  # in seconds (1 ms)
# time between the start of consecutive epochs
event_interval = epoch_length - overlap
# Generate fake events
sfreq = 256  # sampling frequency
event_duration = int(sfreq * epoch_length)  # number of samples in an epoch
# number of samples between consecutive events
event_step = int(sfreq * event_interval)
n_events = int((153344 - event_duration) / event_step) + 1
events = np.array([[int(i * event_step), 0, 1] for i in range(n_events)])
frequencies = np.arange(8, 30, 1)  # Frequencies of interest from 8 to 30 Hz
n_cycles = frequencies / 2.  # Number of cycles in Morlet wavelet
groupName = ['HTR','CTR','VIP','SNY']
groups = ['0','1','2','3']

In [2]:
def compute_tda(types,group_idx,file_idx):
    data_dir = pathlib.Path(f"./data/raw/{types}/")
    rawPath = []
    files = list(data_dir.glob(f"*{groups[group_idx]}_raw.fif"))
    rawPath = sorted(files)
    rawPath = rawPath[file_idx-1:file_idx]
        # create epochs
    epochList = []
    for path in rawPath:
        raw = mne.io.read_raw_fif(path, preload=True)
        epochs = mne.Epochs(raw, events, event_id=1, tmin=0,
                                tmax=epoch_length, baseline=None, preload=True)
        epochList.append(epochs)
        # get complex signals
    times = epochList[0][0].get_data().shape[2]
    num_epochs = len(epochList[0])
    # Precompute the TFR parameters to avoid recalculating them in each iteration
    data = epochList[0].get_data()
            # Precompute the TFR parameters to avoid recalculating them in each iteration
    tfr_params = {
                'sfreq': 256,
                'freqs': frequencies,
                'n_cycles': n_cycles,
                'use_fft': True,
                'output': 'complex',
                'n_jobs': 20
    }
            # Compute complex signals for all time points at once using TFR
    complex_signal = mne.time_frequency.tfr_array_morlet(
                # Add a new axis to match expected input shape (1, n_channels, n_times)
                data,
                **tfr_params
    )
    # calculate plv
    powers = np.zeros((num_epochs, len(frequencies), 64, 64), dtype=np.float64)
    phases = np.zeros((num_epochs, len(frequencies), 64, 64), dtype=np.float64)
    for epoch_idx in range(num_epochs):
        for freq_idx in range(len(frequencies)):
            phaseEpoch = np.angle(complex_signal[epoch_idx, :, freq_idx, :])
            powerEpoch = np.abs(complex_signal[epoch_idx, :, freq_idx, :])
            phaseCorr = 1 - np.corrcoef(phaseEpoch)
            powerCorr = 1 - np.corrcoef(powerEpoch)
            phases[epoch_idx, freq_idx] = phaseCorr
            powers[epoch_idx, freq_idx] = powerCorr
    ############################################################################################
        # Topological data analysis
#         Power
#     persistentEntropyPower = np.zeros(
#             (2, num_epochs, len(frequencies)), dtype=np.float64)
#     amplitudePower = np.zeros(
#             (2, num_epochs, len(frequencies)), dtype=np.float64)
#     bettiAreaPower = np.zeros(
#             (2, num_epochs, len(frequencies)), dtype=np.float64)
#         # Phase
#     persistentEntropyPhase = np.zeros(
#             (2, num_epochs, len(frequencies)), dtype=np.float64)
#     amplitudePhase = np.zeros(
#             (2, num_epochs, len(frequencies)), dtype=np.float64)
#     bettiAreaPhase = np.zeros(
#             (2, num_epochs, len(frequencies)), dtype=np.float64)
#     power_point_cloud = []
    for epoch_idx in range(1):
        for freq_idx, freq in enumerate(frequencies):
                    # Initialize Vietoris-Rips complex
            vr = VietorisRipsPersistence(homology_dimensions=[0, 1, 2])
                    # Ensure powers is correctly structured
            data1 = powers[epoch_idx][freq_idx]
            data2 = phases[epoch_idx][freq_idx]
                    # Fit and transform the data
            diagramsPower = vr.fit_transform([data1])
            diagramsPhase = vr.fit_transform([data2])
            return diagramsPhase
#     return diagramsPhase
                    # Initialize BettiCurve, PersistenceEntropy, and Amplitude
        #     betti = BettiCurve()
        #     pEntropy = PersistenceEntropy()
        #     ampObject = Amplitude()
        #             ################################################################################### Power
        #             # Compute the curves and entropies
        #     betti_curves1 = betti.fit_transform(diagramsPower)
        #     pee1 = pEntropy.fit_transform(diagramsPower)
        #     aa1 = ampObject.fit_transform(diagramsPower)
        #             # Calculate the area under Betti curves
        #     area = []
        #     radii = np.linspace(0, 1, betti_curves1.shape[2])
        #     for i in range(2):
        #         area.append(np.trapz(betti_curves1[0][i], radii))
        #             # Update the power arrays
        #     persistentEntropyPower[0,epoch_idx, freq_idx] = pee1[0][0]
        #     persistentEntropyPower[1,epoch_idx, freq_idx] = pee1[0][1]
        #     amplitudePower[0, epoch_idx, freq_idx] = aa1[0][0]
        #     amplitudePower[1, epoch_idx, freq_idx] = aa1[0][1]
        #     bettiAreaPower[0, epoch_idx, freq_idx] = area[0]
        #     bettiAreaPower[1, epoch_idx, freq_idx] = area[1]
        #             ################################################################################### Phase
        #             # Compute the curves and entropies
        #     betti_curves = betti.fit_transform(diagramsPhase)
        #     pee2 = pEntropy.fit_transform(diagramsPhase)
        #     aa2 = ampObject.fit_transform(diagramsPhase)
                    # Calculate the area under Betti curves
#             area = []
#             radii = np.linspace(0, 1, betti_curves.shape[2])
#             for i in range(2):
#                 area.append(np.trapz(betti_curves[0][i], radii))
#                     # Update the power arrays
#             persistentEntropyPhase[0,epoch_idx, freq_idx] = pee2[0][0]
#             persistentEntropyPhase[1,epoch_idx, freq_idx] = pee2[0][1]
#             amplitudePhase[0, epoch_idx, freq_idx] = aa2[0][0]
#             amplitudePhase[1, epoch_idx, freq_idx] = aa2[0][1]
#             bettiAreaPhase[0, epoch_idx, freq_idx] = area[0]
#             bettiAreaPhase[1, epoch_idx, freq_idx] = area[1]

#     df1 = pd.DataFrame(persistentEntropyPhase[0])
#     df2 = pd.DataFrame(persistentEntropyPhase[1])
#     df3 = pd.DataFrame(amplitudePhase[0])
#     df4 = pd.DataFrame(amplitudePhase[1])
#     df5 = pd.DataFrame(bettiAreaPhase[0])
#     df6 = pd.DataFrame(bettiAreaPhase[1])
#     df7 = pd.DataFrame(persistentEntropyPower[0])
#     df8 = pd.DataFrame(persistentEntropyPower[1])
#     df9 = pd.DataFrame(amplitudePower[0])
#     df10 = pd.DataFrame(amplitudePower[1])
#     df11 = pd.DataFrame(bettiAreaPower[0])
#     df12 = pd.DataFrame(bettiAreaPower[1])
#     subject = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12], axis=0)
#     subject.to_csv(f"./data/48plots/pearsonCorr/features/{types}_group{groupName[group_idx]}-{file_idx}.csv", index=False)

In [3]:
d = compute_tda("think2", 0,1)
d

Opening raw data file data/raw/think2/file0_0_raw.fif...
Isotrak not found
    Range : 0 ... 153343 =      0.000 ...   598.996 secs
Ready.


Reading 0 ... 153343  =      0.000 ...   598.996 secs...
Not setting metadata
595 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 595 events and 1281 original time points ...
1 bad epochs dropped


/tmp/ipykernel_1309416/1225501496.py:15: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  times = epochList[0][0].get_data().shape[2]
/tmp/ipykernel_1309416/1225501496.py:18: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data = epochList[0].get_data()
/tmp/ipykernel_1309416/1225501496.py:29: FutureWarning: The default value of `zero_mean` will change from `False` to `True` in version 1.8. Set the value explicitly to avoid this warning.
  complex_signal = mne.time_frequency.tfr_array_morlet(
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  58 out of  64 | elapsed:   46.1s remaining:    4.8s
[Parallel(n_jobs=20)]: Done  64 out of  64 | elapsed:   49.8s finished
/home/server/anaconda3/envs/work/lib/python3.12/site-packages/gtda/homology/simplicial.py:23

array([[[0.        , 0.14414196, 0.        ],
        [0.        , 0.15077391, 0.        ],
        [0.        , 0.15332858, 0.        ],
        [0.        , 0.1704753 , 0.        ],
        [0.        , 0.17964539, 0.        ],
        [0.        , 0.18190424, 0.        ],
        [0.        , 0.19743189, 0.        ],
        [0.        , 0.22143289, 0.        ],
        [0.        , 0.22397912, 0.        ],
        [0.        , 0.23658787, 0.        ],
        [0.        , 0.23771423, 0.        ],
        [0.        , 0.24348968, 0.        ],
        [0.        , 0.24741684, 0.        ],
        [0.        , 0.25090161, 0.        ],
        [0.        , 0.25129035, 0.        ],
        [0.        , 0.2567203 , 0.        ],
        [0.        , 0.25748494, 0.        ],
        [0.        , 0.26217058, 0.        ],
        [0.        , 0.27049285, 0.        ],
        [0.        , 0.28759217, 0.        ],
        [0.        , 0.28784895, 0.        ],
        [0.        , 0.29568911, 0

In [6]:
from gtda.plotting import plot_point_cloud
plot_point_cloud(d[0])

In [39]:
channel_name = 'F1'
picks = mne.pick_channels(d[0].info['ch_names'], include=[channel_name]) 

In [45]:
d[0].info

<Info | 9 non-empty values
 bads: []
 ch_names: Fp1, AF7, AF3, F1, F3, F5, F7, FT7, FC5, FC3, FC1, C1, C3, C5, ...
 chs: 64 EEG
 custom_ref_applied: False
 file_id: 4 items (dict)
 highpass: 8.0 Hz
 lowpass: 30.0 Hz
 meas_date: 2009-04-01 12:03:04 UTC
 meas_id: 4 items (dict)
 nchan: 64
 projs: []
 sfreq: 256.0 Hz
>

In [50]:
ok = d[0].get_data()

/tmp/ipykernel_631453/1676636205.py:1: FutureWarning:

The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning



In [54]:
picks[0]

3

In [67]:
data = d[0].get_data()[:,picks, :]

/tmp/ipykernel_631453/3620638131.py:1: FutureWarning:

The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning



In [61]:
from gtda.time_series import SingleTakensEmbedding
embedding_dimension = 30
embedding_time_delay = 100
stride = 10

embedder = SingleTakensEmbedding(
    parameters_type="search", n_jobs=6, time_delay=embedding_time_delay, dimension=embedding_dimension, stride=stride
)

In [68]:
data = data[0,0,:]
data.shape

(1281,)

In [69]:
y_signal_embedded = embedder.fit_transform(data)

In [70]:
from sklearn.decomposition import PCA
from gtda.plotting import plot_point_cloud

pca = PCA(n_components=3)
y_gw_embedded_pca = pca.fit_transform(y_signal_embedded)

plot_point_cloud(y_gw_embedded_pca)